In [6]:
# code from : 
# https://machinelearningmastery.com/how-to-develop-rnn-models-for-human-activity-recognition-time-series-classification/
# lstm model
from numpy import mean
from numpy import std
from numpy import dstack
from numpy import reshape
from pandas import read_csv
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import GRU
from keras.utils import to_categorical
from matplotlib import pyplot
import numpy as np
import gc
import asyncio
from time import gmtime, strftime
import sklearn
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
import matplotlib.pyplot as plt

In [7]:
import tensorflow as tf 
from keras.backend.tensorflow_backend import set_session
config =  tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True  # dynamically grow the memory used on the GPU
config.log_device_placement = True  # to log device placement (on which device the operation ran)
                                    # (nothing gets printed in Jupyter, only if you run it standalone)
sess = tf.compat.v1.Session(config=config)
tf.compat.v1.keras.backend.set_session(sess)
 # set this TensorFlow session as the default session for Keras

Device mapping:
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: GeForce RTX 2080 Ti, pci bus id: 0000:01:00.0, compute capability: 7.5



In [8]:
%%javascript

IPython.tab_as_tab_everywhere = function(use_tabs) {
    if (use_tabs === undefined) {
        use_tabs = true; 
    }

    // apply setting to all current CodeMirror instances
    IPython.notebook.get_cells().map(
        function(c) {  return c.code_mirror.options.indentWithTabs=use_tabs;  }
    );
    // make sure new CodeMirror instances created in the future also use this setting
    CodeMirror.defaults.indentWithTabs=use_tabs;

    };

IPython.tab_as_tab_everywhere()

<IPython.core.display.Javascript object>

In [9]:
def load_dataset2(prefix='C:/Users/alame/OneDrive - unige.it/Documents/TactileDataset/cleand (6144)/(64x64) time frames(images)/RGB/Features/'):
	#X1 = read_csv(prefix+'rolling/'+'rolling_all_t1_t24.txt', header=None, delim_whitespace=True) 
	#X2 = read_csv(prefix+'sliding/'+'sliding_all_t1_t24.txt', header=None, delim_whitespace=True) 
	X1 = read_csv(prefix+'rolling/'+'rolling_all_t1_t24.txt', header=None, delim_whitespace=True) 
	X2 = read_csv(prefix+'sliding/'+'sliding_all_t1_t24.txt', header=None, delim_whitespace=True) 
	X3 = read_csv(prefix+'brushing/'+'brushing_all_t1_t24.txt', header=None, delim_whitespace=True) 
	n_features = 8192
	#A=X1.values
	#B=X2.values
	Xlen= X1.shape[0]
	X1=X1.values
	X2=X2.values
	X3=X3.values
	
	X1_train = X1[0:4*Xlen//5,:]
	X2_train = X2[0:4*Xlen//5,:]
	X3_train = X3[0:4*Xlen//5,:]
	
	X_train = np.zeros((X1_train.shape[0]*2,X1_train.shape[1]))
	X_train [ 0:X1_train.shape[0],:] = X1_train
	X_train [X1_train.shape[0]:X1_train.shape[0]*2,:]=X2_train
	
	X1_test = X1[4*Xlen//5:Xlen,:]
	X2_test = X2[4*Xlen//5:Xlen,:]
	X_test = np.zeros((X1_test.shape[0]*2,X1_test.shape[1]))
	X_test[ 0:X1_test.shape[0],:] = X1_test
	X_test [X1_test.shape[0]:X1_test.shape[0]*2,:]=X2_test
	#X= np.concatenate (A,B,1)
	#print (np.shape(X1))
	#print (np.shape(X2))
	Y = read_csv(prefix + 'y.txt', header=None, delim_whitespace=True)
	Y=Y.values
	Y=to_categorical(Y)
	Ylen= Y.shape[0]
	Y1 = Y[0:Ylen//2,:]
	Y2 = Y[Ylen//2:Ylen,:]
	Y1_train = Y1[0:4*Ylen//5//2,:]
	Y2_train = Y2[0:4*Ylen//5//2,:]
	Y_train = np.zeros((Y1_train.shape[0]*2,Y1_train.shape[1]))
	Y_train [ 0:Y1_train.shape[0],:] = Y1_train
	Y_train [Y1_train.shape[0]:Y1_train.shape[0]*2,:]=Y2_train
	#Y_train = Y1_train.append(Y2_train)
	Y1_test = Y1[4*Ylen//5//2:Ylen//2,:]
	Y2_test = Y2[4*Ylen//5//2:Ylen//2,:]
	Y_test = np.zeros((Y1_test.shape[0]*2,Y1_test.shape[1]))
	Y_test [ 0:Y1_test.shape[0],:] = Y1_test
	Y_test [Y1_test.shape[0]:Y1_test.shape[0]*2,:]=Y2_test
	timelength = 24
	X_train= np.reshape((X_train),(X_train.shape[0]//timelength,-1,n_features))
	X_test= np.reshape((X_test),(X_test.shape[0]//timelength,-1,n_features))
	#Y_train= np.reshape((Y_train),(Y_train.shape[0]//timelength,-1,8192))
	#Y_test= np.reshape((Y_test),(Y_test.shape[0]//timelength,-1,8192))
	return X_train, Y_train, X_test ,Y_test 







In [10]:
def load_dataset3(prefix='C:/Users/alame/OneDrive - unige.it/Documents/TactileDataset/cleand (6144)/(64x64) time frames(images)/RGB/Features/Features(ResNet150-v2)/'):

# 	X1 = read_csv(prefix+'rolling/'+'rolling_all_t1_t24.txt', header=None, delim_whitespace=True) 
# 	X2 = read_csv(prefix+'sliding/'+'sliding_all_t1_t24.txt', header=None, delim_whitespace=True) 
# 	X3 = read_csv(prefix+'brushing/'+'brushing_all_t1_t24.txt', header=None, delim_whitespace=True) 
	X1 = read_csv(prefix+'rolling/'+'rolling_all_resnet150.txt', header=None, delim_whitespace=True) 
	X2 = read_csv(prefix+'sliding/'+'sliding_all_resnet150.txt', header=None, delim_whitespace=True) 
	X3 = read_csv(prefix+'brushing/'+'brushing_all_resnet150.txt', header=None, delim_whitespace=True) 

	n_features = 8192
	timelength = 24

	#A=X1.values
	#B=X2.values
	Xlen= X1.shape[0]
	X1=X1.values
	X2=X2.values
	X3=X3.values
	
	X1_train = X1[0:4*Xlen//5,:]
	X2_train = X2[0:4*Xlen//5,:]
	X3_train = X3[0:4*Xlen//5,:]
	
	X_train = np.zeros((X1_train.shape[0]*3,X1_train.shape[1]))
	X_train [ 0:X1_train.shape[0],:] = X1_train
	X_train [X1_train.shape[0]:X1_train.shape[0]*2,:]=X2_train
	X_train [X1_train.shape[0]*2:X1_train.shape[0]*3,:]=X3_train
	
	X1_test = X1[4*Xlen//5:Xlen,:]
	X2_test = X2[4*Xlen//5:Xlen,:]
	X3_test = X3[4*Xlen//5:Xlen,:]
	
	X_test = np.zeros((X1_test.shape[0]*3,X1_test.shape[1]))
	X_test[ 0:X1_test.shape[0],:] = X1_test
	X_test [X1_test.shape[0]:X1_test.shape[0]*2,:]=X2_test
	X_test [X1_test.shape[0]*2:X1_test.shape[0]*3,:]=X3_test
	
	#X= np.concatenate (A,B,1)
	#print (np.shape(X1))
	#print (np.shape(X2))
	Y = read_csv(prefix + 'y.txt', header=None, delim_whitespace=True)
	Y = Y.values
	Y = to_categorical(Y)
	Ylen = Y.shape[0]
	Y1 = Y[0:Ylen//3,:]
	Y2 = Y[Ylen//3:2*Ylen//3,:]
	Y3 = Y[2*Ylen//3:Ylen,:]


	Y1_train = Y1[0:4*Ylen//5//3,:]
	Y2_train = Y2[0:4*Ylen//5//3,:]
	Y3_train = Y3[0:4*Ylen//5//3,:]

	Y_train = np.zeros((Y1_train.shape[0]*3,Y1_train.shape[1]))

	Y_train [ 0:Y1_train.shape[0],:] = Y1_train
	Y_train [Y1_train.shape[0]:Y1_train.shape[0]*2,:] = Y2_train
	Y_train [Y1_train.shape[0]*2:Y1_train.shape[0]*3,:] = Y3_train

	#Y_train = Y1_train.append(Y2_train)
	Y1_test = Y1[4*Ylen//5//3:Ylen//3,:]
	Y2_test = Y2[4*Ylen//5//3:Ylen//3,:]
	Y3_test = Y3[4*Ylen//5//3:Ylen//3,:]

	Y_test = np.zeros((Y1_test.shape[0]*3,Y1_test.shape[1]))
	Y_test [ 0:Y1_test.shape[0],:] = Y1_test
	Y_test [Y1_test.shape[0]:Y1_test.shape[0]*2,:]=Y2_test
	Y_test [Y1_test.shape[0]*2:Y1_test.shape[0]*3,:]=Y3_test

	X_train= np.reshape((X_train),(X_train.shape[0]//timelength,-1,n_features))
	#X_train= np.reshape((X_train),(X_train.shape[0],16,256))
	X_test= np.reshape((X_test),(X_test.shape[0]//timelength,-1,n_features))
	#X_test= np.reshape((X_test),(X_test.shape[0],16,256))
	
	#Y_train= np.reshape((Y_train),(Y_train.shape[0]//timelength,-1,8192))
	#Y_test= np.reshape((Y_test),(Y_test.shape[0]//timelength,-1,8192))
	return X_train, Y_train, X_test ,Y_test 







In [11]:
def load_dataset4 (prefix='C:/Users/alame/OneDrive - unige.it/Documents/TactileDataset/cleand (6144)/Average in time/output=20/'):

# (prefix='C:/Users/alame/OneDrive - unige.it/Documents/TactileDataset/cleand (6144)/(64x64) time frames(images)/RGB/Features/Features(ResNet150-v2)/'):

# 	X1 = read_csv(prefix+'rolling/'+'rolling_all_resnet150.txt', header=None, delim_whitespace=True) 
# 	X2 = read_csv(prefix+'sliding/'+'sliding_all_resnet150.txt', header=None, delim_whitespace=True) 
# 	X3 = read_csv(prefix+'brushing/'+'brushing_all_resnet150.txt', header=None, delim_whitespace=True) 
	#(prefix='C:/Users/alame/OneDrive - unige.it/Documents/TactileDataset/cleand (6144)/Average in time/output=256/Dataset/'):
	#
# 	X1 = read_csv(prefix+'rolling/'+'rolling_all_t1_t24.txt', header=None, delim_whitespace=True) 
# 	X2 = read_csv(prefix+'sliding/'+'sliding_all_t1_t24.txt', header=None, delim_whitespace=True) 
# 	X3 = read_csv(prefix+'brushing/'+'brushing_all_t1_t24.txt', header=None, delim_whitespace=True) 
	X1 = read_csv(prefix+'rolling/'+'rolling_all_20.txt', header=None, delim_whitespace=False) 
	X2 = read_csv(prefix+'sliding/'+'sliding_all_20.txt', header=None, delim_whitespace=False) 
	X3 = read_csv(prefix+'brushing/'+'brushing_all_20.txt', header=None, delim_whitespace=False) 
	print (prefix)

	mu = np.average([np.average(X1),np.average(X2),np.average(X2)])  ##### here there was a mistake the last X2 should be X3
	std = np.std([np.std(X1),np.std(X2),np.std(X2)]) 
	n_features = 16#8192#
	timelength = 20#24#200 #24

	#A=X1.values
	#B=X2.values
	Xlen= X1.shape[0]
	X1=X1.values
	X2=X2.values
	X3=X3.values
	
	X1_train = X1[0:4*Xlen//5,:]
	X2_train = X2[0:4*Xlen//5,:]
	X3_train = X3[0:4*Xlen//5,:]
	
	X_train = np.zeros((X1_train.shape[0]*3,X1_train.shape[1]))
	X_train [ 0:X1_train.shape[0],:] = X1_train
	X_train [X1_train.shape[0]:X1_train.shape[0]*2,:]=X2_train
	X_train [X1_train.shape[0]*2:X1_train.shape[0]*3,:]=X3_train
	
	X1_test = X1[4*Xlen//5:Xlen,:]
	X2_test = X2[4*Xlen//5:Xlen,:]
	X3_test = X3[4*Xlen//5:Xlen,:]
	
	X_test = np.zeros((X1_test.shape[0]*3,X1_test.shape[1]))
	X_test[ 0:X1_test.shape[0],:] = X1_test
	X_test [X1_test.shape[0]:X1_test.shape[0]*2,:]=X2_test
	X_test [X1_test.shape[0]*2:X1_test.shape[0]*3,:]=X3_test
	
	#X= np.concatenate (A,B,1)
	#print (np.shape(X1))
	#print (np.shape(X2))
	Y = read_csv(prefix + 'y.txt', header=None, delim_whitespace=True)
	Y = Y.values
	Y = to_categorical(Y)
	Ylen = Y.shape[0]
	Y1 = Y[0:Ylen//3,:]
	Y2 = Y[Ylen//3:2*Ylen//3,:]
	Y3 = Y[2*Ylen//3:Ylen,:]


	Y1_train = Y1[0:4*Ylen//5//3,:]
	Y2_train = Y2[0:4*Ylen//5//3,:]
	Y3_train = Y3[0:4*Ylen//5//3,:]

	Y_train = np.zeros((Y1_train.shape[0]*3,Y1_train.shape[1]))

	Y_train [ 0:Y1_train.shape[0],:] = Y1_train
	Y_train [Y1_train.shape[0]:Y1_train.shape[0]*2,:] = Y2_train
	Y_train [Y1_train.shape[0]*2:Y1_train.shape[0]*3,:] = Y3_train

	#Y_train = Y1_train.append(Y2_train)
	Y1_test = Y1[4*Ylen//5//3:Ylen//3,:]
	Y2_test = Y2[4*Ylen//5//3:Ylen//3,:]
	Y3_test = Y3[4*Ylen//5//3:Ylen//3,:]

	Y_test = np.zeros((Y1_test.shape[0]*3,Y1_test.shape[1]))
	Y_test [ 0:Y1_test.shape[0],:] = Y1_test
	Y_test [Y1_test.shape[0]:Y1_test.shape[0]*2,:]=Y2_test
	Y_test [Y1_test.shape[0]*2:Y1_test.shape[0]*3,:]=Y3_test

	X_train= np.reshape((X_train),(X_train.shape[0]//timelength,-1,n_features))
	X_train = (X_train - mu )/ std
	

	X_test= np.reshape((X_test),(X_test.shape[0]//timelength,-1,n_features))
	X_test = (X_test - mu )/ std
	#Y_train= np.reshape((Y_train),(Y_train.shape[0]//timelength,-1,8192))
	#Y_test= np.reshape((Y_test),(Y_test.shape[0]//timelength,-1,8192))
	return X_train, Y_train, X_test ,Y_test 

###load_dataset4





In [12]:
print ("start Loading dataset .....")
myDataset= load_dataset4()
# run the experiment
print ("dataset Loaded")


start Loading dataset .....
C:/Users/alame/OneDrive - unige.it/Documents/TactileDataset/cleand (6144)/Average in time/output=20/
dataset Loaded


In [13]:
def load_dataset5 (fold = 0,prefix=''):
	#print (prefix)


	X1 = read_csv(prefix+'rolling/'+'rolling_all.txt', header=None, delim_whitespace=False) 
	X2 = read_csv(prefix+'sliding/'+'sliding_all.txt', header=None, delim_whitespace=False) 
	X3 = read_csv(prefix+'brushing/'+'brushing_all.txt', header=None, delim_whitespace=False) 
	print (prefix)

	mu = np.average([np.average(X1),np.average(X2),np.average(X3)]) 
	std = np.std([np.std(X1),np.std(X2),np.std(X3)]) 
	n_features = 768# tactnet2850 896#8192#2048#
	timelength = 24#20#200 

	#A=X1.values
	#B=X2.values
	#Xlen= X1.shape[0]
	X1=X1.values
	X2=X2.values
	X3=X3.values

	X = np.zeros((X1.shape[0]*3,X1.shape[1]))
	X [ 0:X1.shape[0],:] = X1
	X [X1.shape[0]:X1.shape[0]*2,:]=X2
	X [X1.shape[0]*2:X1.shape[0]*3,:]=X3
	#X = (X-mu)/std ; print (" with regularization"); 
	print ("without regularization")
	X = np.reshape((X),(X.shape[0]//timelength,-1,n_features))


	Y = read_csv(prefix + 'y.txt', header=None, delim_whitespace=True)
	Y = Y.values
	y= Y
	#y = to_categorical(Y)
	#X,y = shuffle(X,y)
	skf = StratifiedKFold(n_splits=5,shuffle = False,random_state=None)
	#print (skf.get_n_splits(X, y))
	lst = list(skf.split(X,y))
	train_index, test_index  = list(skf.split(X,y))[fold]
	return X[train_index],to_categorical( y[train_index]), X[test_index], to_categorical(y[test_index])
	
	
# 	for train_index, test_index in skf.split(X, y):
# 		#print("TRAIN:", train_index, "TEST:", test_index)
# 		X_train, X_test = X[train_index], X[test_index]
# 		y_train, y_test = y[train_index], y[test_index]
# 		#print("Train and test" ,X_train , X_test)
# 		#print ("\r\n")
# 		X_train, X_val,y_train,y_val =train_test_split(X_train, y_train,train_size = 0.75 ,shuffle=False)
# 		print ("tr , val",y_train,y_val)
   

In [ ]:
# ALAME This test failed with our data  classified as 320 per row , accuracy 49 - 50 %
# load a single file as a numpy array
# I will retry using the new method 
# same was on the new method, making a Standardization works fine


# fit and evaluate a model
def evaluate_model(trainX, trainy, testX, testy,epochs,batch_size):
	#verbose, epochs, batch_size = 1, 56, 100
	#number of parameters of a single LSTM layer: (4 * ((size_of_input + 1) * size_of_output + size_of_output^2))
	verbose = 0
	n_timesteps, n_features, n_outputs = trainX.shape[1], trainX.shape[2], trainy.shape[1]
	#print ("features " , n_features)
	#print ("Outputs " , n_outputs)
	model = Sequential()
# 	model.add(LSTM(100, input_shape=(n_timesteps,n_features),return_sequences=True)) ;print ("first lstm layer 100 neurons") # original 100
	model.add(LSTM(10, input_shape=(n_timesteps,n_features))); print ("LSTM layer 10 neurons") # original 100 ; 
	#model.add(Dropout(0.5))
	#model.add(Dense(10, activation='relu')) # original 200
	model.add(Dense(n_outputs, activation='softmax'))
	#model.summary()
	model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
	# fit network
	history = model.fit(trainX, trainy,  epochs=epochs, batch_size=batch_size, verbose=verbose)
	#history = model.fit(trainX, trainy, validation_split=0.25, epochs=epochs, batch_size=batch_size, verbose=verbose) # it seems worse with validation
#	print (history.history.keys())
# 	plt.plot(history.history['accuracy'])
# 	#plt.plot(history.history['val_accuracy'])
# 	plt.title('Model accuracy')
# 	plt.ylabel('Accuracy')
# 	plt.xlabel('Epoch')
# 	plt.legend(['Train', 'Val'], loc='upper left')
# 	plt.show()

# 	# Plot training & validation loss values
# 	plt.plot(history.history['loss'])
# 	#plt.plot(history.history['val_loss'])
# 	plt.title('Model loss')
# 	plt.ylabel('Loss')
# 	plt.xlabel('Epoch')
# 	plt.legend(['Train', 'Val'], loc='upper left')
# 	plt.show()

	# evaluate model
	_, accuracy = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
	print ( "verbose ",verbose, " epochs ",epochs, " batch_size ",batch_size)
	return accuracy
    


def evaluate_model_GRU(trainX, trainy, testX, testy,epochs,batch_size):

	verbose = 0
	n_timesteps, n_features, n_outputs = trainX.shape[1], trainX.shape[2], trainy.shape[1]

	model = Sequential()

	model.add(GRU(12, input_shape=(n_timesteps,n_features))); print ("GRU layer 12 neurons") # original 100 ; 

	model.add(Dense(n_outputs, activation='softmax'))
	#model.summary()
	model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
	# fit network
	history = model.fit(trainX, trainy,  epochs=epochs, batch_size=batch_size, verbose=verbose)
# 	#history = model.fit(trainX, trainy, validation_split=0.25, epochs=epochs, batch_size=batch_size, verbose=verbose) # it seems worse with validation
# 	print (history.history.keys())
# 	plt.plot(history.history['accuracy'])
# 	#plt.plot(history.history['val_accuracy'])
# 	plt.title('Model accuracy')
# 	plt.ylabel('Accuracy')
# 	plt.xlabel('Epoch')
# 	plt.legend(['Train', 'Val'], loc='upper left')
# 	plt.show()

# 	# Plot training & validation loss values
# 	plt.plot(history.history['loss'])
# 	#plt.plot(history.history['val_loss'])
# 	plt.title('Model loss')
# 	plt.ylabel('Loss')
# 	plt.xlabel('Epoch')
# 	plt.legend(['Train', 'Val'], loc='upper left')
# 	plt.show()

	# evaluate model
	_, accuracy = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
	print ( "verbose ",verbose, " epochs ",epochs, " batch_size ",batch_size)
	return accuracy

# summarize scores
def summarize_results(scores):
	print(scores)
	m, s = mean(scores), std(scores)
	print('Accuracy: %.3f%% (+/-%.3f)' % (m, s))

# run an experiment
def run_experiment(myDataset,epochs,batch_size,repeats=3):
	# load data
	trainX, trainy, testX, testy = myDataset
	# repeat experiment
	scores = list()
	for r in range(repeats):
		score = evaluate_model(trainX, trainy, testX, testy,epochs,batch_size)
		#score = evaluate_model(trainX, trainy, testX, testy,epochs,batch_size)
		score = score * 100.0
		#print('>#%d: %.3f' % (r+1, score), strftime("%d/%m/%Y %H:%M:%S +0000", gmtime()))
		scores.append(score)
	# summarize results
	summarize_results(scores)
	print(strftime("%d/%m/%Y %H:%M:%S +0000", gmtime()))

print ("starting ......")
gc.enable()
#for i in [124]:
for  ff in range(0,5):
	#myDataset = load_dataset5(fold = ff, prefix = 'C:/Users/alame/OneDrive - unige.it/Documents/TactileDataset/cleand (6144)/Average in time/output=20/')
	myDataset = load_dataset5(fold = ff, prefix = 'C:/Users/alame/OneDrive - unige.it/Documents/TactileDataset/cleand (6144)/TactNetFeatures/26x47/')
	print ("starting Fold ", ff )
	for i in [48,96]:
		for j in [48,96]:
		#for j in [96]:
			print ("Fold ", ff, "epochs:",i,", batch: ",j)
			run_experiment(myDataset,epochs=i, batch_size=j,repeats = 10)
			gc.collect()
print ("finished!")

In [4]:
model.summary
size_of_input = 16
size_of_output = 3
params = 4 * ((size_of_input + 1) * size_of_output + size_of_output^2)

NameError: name 'model' is not defined

In [5]:
model = Sequential()

model.add(GRU(12, input_shape=(20,16))); print  # original 100 ; 

model.add(Dense(3, activation='softmax'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru_1 (GRU)                  (None, 12)                1044      
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 39        
Total params: 1,083
Trainable params: 1,083
Non-trainable params: 0
_________________________________________________________________


In [9]:
import matplotlib.pyplot as plt
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()



NameError: name 'history' is not defined

In [ ]:
%%javascript
%% this code to enable the right indentation when pressing Tab , Enter
IPython.tab_as_tab_everywhere = function(use_tabs) {
    if (use_tabs === undefined) {
        use_tabs = true; 
    }

    // apply setting to all current CodeMirror instances
    IPython.notebook.get_cells().map(
        function(c) {  return c.code_mirror.options.indentWithTabs=use_tabs;  }
    );
    // make sure new CodeMirror instances created in the future also use this setting
    CodeMirror.defaults.indentWithTabs=use_tabs;

    };

IPython.tab_as_tab_everywhere()

In [ ]:
for i in [1,15,16]:
	for j in [20,50,40]:
		print (i,"",j)
		
		

X_train, Y_train, X_test ,Y_test =myDataset 
print (X_train.shape ,Y_train.shape,X_test.shape, Y_test.shape)
#print (X_test)

mu = np.average([np.average(X_train),np.average(X_test)]) 
import matplotlib as mplib
mplib.plot(X_train[0,:,:])


In [62]:
prefix='C:/Users/alame/OneDrive - unige.it/Documents/TactileDataset/cleand (6144)/Average in time/output=256/Dataset/'
#
X1 = read_csv(prefix+'rolling/'+'rolling_all_256.txt', header=None, delim_whitespace=False) 
X2 = read_csv(prefix+'sliding/'+'sliding_all_256.txt', header=None, delim_whitespace=False) 
X3 = read_csv(prefix+'brushing/'+'brushing_all_256.txt', header=None, delim_whitespace=False) 
n_features = 16
timelength = 256
#A=X1.values
#B=X2.values
Xlen= X1.shape[0]
X1=X1.values
X2=X2.values
X3=X3.values

X1_train = X1[0:4*Xlen//5,:]
X2_train = X2[0:4*Xlen//5,:]
X3_train = X3[0:4*Xlen//5,:]

X_train = np.zeros((X1_train.shape[0]*3,X1_train.shape[1]))
X_train [ 0:X1_train.shape[0],:] = X1_train
X_train [X1_train.shape[0]:X1_train.shape[0]*2,:]=X2_train
X_train [X1_train.shape[0]*2:X1_train.shape[0]*3,:]=X3_train

X1_test = X1[4*Xlen//5:Xlen,:]
X2_test = X2[4*Xlen//5:Xlen,:]
X3_test = X3[4*Xlen//5:Xlen,:]

X_test = np.zeros((X1_test.shape[0]*3,X1_test.shape[1]))
X_test[ 0:X1_test.shape[0],:] = X1_test
X_test [X1_test.shape[0]:X1_test.shape[0]*2,:]=X2_test
X_test [X1_test.shape[0]*2:X1_test.shape[0]*3,:]=X3_test

#X= np.concatenate (A,B,1)
#print (np.shape(X1))
#print (np.shape(X2))
Y = read_csv(prefix + 'y.txt', header=None, delim_whitespace=True)
Y = Y.values
Y = to_categorical(Y)
Ylen = Y.shape[0]
Y1 = Y[0:Ylen//3,:]
Y2 = Y[Ylen//3:2*Ylen//3,:]
Y3 = Y[2*Ylen//3:Ylen,:]


Y1_train = Y1[0:4*Ylen//5//3,:]
Y2_train = Y2[0:4*Ylen//5//3,:]
Y3_train = Y3[0:4*Ylen//5//3,:]

Y_train = np.zeros((Y1_train.shape[0]*3,Y1_train.shape[1]))

Y_train [ 0:Y1_train.shape[0],:] = Y1_train
Y_train [Y1_train.shape[0]:Y1_train.shape[0]*2,:] = Y2_train
Y_train [Y1_train.shape[0]*2:Y1_train.shape[0]*3,:] = Y3_train

#Y_train = Y1_train.append(Y2_train)
Y1_test = Y1[4*Ylen//5//3:Ylen//3,:]
Y2_test = Y2[4*Ylen//5//3:Ylen//3,:]
Y3_test = Y3[4*Ylen//5//3:Ylen//3,:]

Y_test = np.zeros((Y1_test.shape[0]*3,Y1_test.shape[1]))
Y_test [ 0:Y1_test.shape[0],:] = Y1_test
Y_test [Y1_test.shape[0]:Y1_test.shape[0]*2,:]=Y2_test
Y_test [Y1_test.shape[0]*2:Y1_test.shape[0]*3,:]=Y3_test

X_train= np.reshape((X_train),(X_train.shape[0]//timelength,256,n_features))
X_test= np.reshape((X_test),(X_test.shape[0]//timelength,256,n_features))
#Y_train= np.reshape((Y_train),(Y_train.shape[0]//timelength,-1,8192))
#Y_test= np.reshape((Y_test),(Y_test.shape[0]//timelength,-1,8192))
#return X_train, Y_train, X_test ,Y_test 










